In [ ]:
# stdlib
# import os

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# use_live_bigquery = False
# os.environ["TEST_BIGQUERY_APIS_LIVE"] = str(use_live_bigquery)

In [ ]:
# stdlib
import os

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

In [ ]:
# third party

# syft absolute
import syft as sy

In [ ]:
# third party
# run email server
from helpers import EmailServer
from helpers import SMTPTestServer

email_server = EmailServer()
email_server.reset_emails()
smtp_server = SMTPTestServer(email_server)
smtp_server.start()

In [ ]:
SERVER_PORT = "8080"
SERVER_URL = f"http://localhost:{SERVER_PORT}"

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    port=SERVER_PORT,
    n_consumers=4,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
# third party
from helpers import load_users

In [ ]:
users = load_users(high_client)

# Create jobs

In [ ]:
# third party
from job_helpers import TestJob
from job_helpers import create_jobs
from job_helpers import extract_code_path

# Test queries

In [ ]:
# maybe we should assign one of each first, and then randomly generate the rest
n_per_user = 2

jobs = create_jobs(users, n_per_user=n_per_user)

In [ ]:
jobs

In [ ]:
# third party
from job_helpers import save_jobs

In [ ]:
print(f"num jobs: {len(jobs)}")
num_should_submit = 0
for job in jobs:
    print(
        f"Job type: {job.job_type}, should succeed: {job.should_succeed}, should submit: {job.should_submit}"
    )
    if job.should_submit:
        num_should_submit += 1

In [ ]:
assert len(jobs) == len(users) * n_per_user
assert all(isinstance(j, TestJob) for j in jobs)
assert all(job.client is not None for job in jobs)

In [ ]:
save_jobs(jobs)

# Submit jobs


In [ ]:
admin_emails_before = len(email_server.get_emails_for_user("admin@bigquery.org"))
print("admin emails before", admin_emails_before)
admin_emails_before

In [ ]:
# stdlib

responses = []

for job in jobs:
    client = job.client

    if not job.should_submit:
        # Submitting should throw error (eg func_name invalid syntax)
        with sy.raises(sy.SyftException):
            response = client.api.services.bigquery.submit_query(
                func_name=job.func_name, query=job.query
            )
        responses.append(None)
    else:
        response = client.api.services.bigquery.submit_query(
            func_name=job.func_name, query=job.query
        )
        job.code_path = extract_code_path(response)
        responses.append(response)

    # time.sleep(1)

In [ ]:
for job in jobs:
    print(
        f"Job {job.func_name:.20} is submitted, {job.should_submit}, should be submitted {job.is_submitted}"
    )

assert all(job.is_submitted == job.should_submit for job in jobs)

In [ ]:
save_jobs(jobs)

## Test: cannot execute

In [ ]:
submitted_jobs = [job for job in jobs if job.should_submit]

assert len(submitted_jobs)  # failsafe for next tests

In [ ]:
# Blocking

for job in submitted_jobs:
    execute_code_fn = getattr(job.client.code, job.code_path)
    with sy.raises(
        sy.SyftException(public_message="*Your code is waiting for approval*")
    ):
        result = execute_code_fn()

In [ ]:
# Nonblocking

# syft absolute
from syft.service.job.job_stash import JobStatus

for job in submitted_jobs:
    execute_code_fn = getattr(job.client.code, job.code_path)
    result_job = execute_code_fn(blocking=False)
    result_job.wait()
    assert isinstance(result_job.result, sy.SyftError)
    assert result_job.status == JobStatus.ERRORED
    # time.sleep(1)

In [ ]:
admin_emails_after = len(email_server.get_emails_for_user("admin@bigquery.org"))
print("admin emails after", admin_emails_after)
assert admin_emails_after >= admin_emails_before + num_should_submit
# assert len(users_emails) > after_number_of_emails
# assert len(users_emails) == after_number_of_emails + 1

In [ ]:
smtp_server.stop()

In [ ]:
server.land()